# OPTaaS - Advanced Options

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:charles.brecque@mindfoundry.ai">here</a>.

## Connect to the OPTaaS server using your API Key

In [1]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Store additional data in a Task

This can be a JSON, array, string, number or boolean:

In [2]:
from mindfoundry.optaas.client.parameter import FloatParameter

task = client.create_task(
    title='My Task with User-defined Data', 
    parameters=[
        FloatParameter('x', minimum=0, maximum=5),
        FloatParameter('y', minimum=1, maximum=5),
    ],
    user_defined_data={
        'description': 'Lorem ipsum...',
        'tags': ['abc', 'defg']
    }
)

print(task.user_defined_data)

{'description': 'Lorem ipsum...', 'tags': ['abc', 'defg']}


## Warm start

If you've already tried some configurations, you can record the scores upfront, thus giving the optimizer a "warm start":

In [3]:
def scoring_function(x, y):
    return (x * y) - (x / y)

warm_start_values = [
    {'x': 0, 'y': 1},
    {'x': 1, 'y': 2},
    {'x': 0.5, 'y': 1.5},
]
for values in warm_start_values:
    task.add_user_defined_configuration(values, scoring_function(**values))

task.get_results()

[{ 'configuration': 'd85df0bf-0b50-4279-ac4e-9b6fda22d749',
   'id': 4457,
   'score': 0.0,
   'user_defined_data': None},
 { 'configuration': 'fda7e52d-220d-406b-98fc-8ac12b70b4f8',
   'id': 4458,
   'score': 1.5,
   'user_defined_data': None},
 { 'configuration': '8326c3e2-0942-4757-8da5-1a5a976d3df5',
   'id': 4459,
   'score': 0.4166666666666667,
   'user_defined_data': None}]

## Store additional data in a Result

This can be a JSON, array, string, number or boolean:

In [4]:
configuration = task.generate_configurations()[0]
score = scoring_function(**configuration.values)
task.record_result(configuration, score, user_defined_data={'Any data': ['you', 'like']})

task.get_results()[-1:]

[{ 'configuration': '343d76e9-2a8d-46c7-8722-979920efb11a',
   'id': 4460,
   'score': 0.6744889678888644,
   'user_defined_data': {'Any data': ['you', 'like']}}]

## Report an error

If you encountered an error while calculating the score for a configuration, you can report it:

In [5]:
error_configuration = task.generate_configurations()[0]
next_configuration = task.record_result(error_configuration, 
                                        error='Unexpected error: code 12345')
error_result = task.get_results()[-1]
print(f'Score: {error_result.score}  {error_result.error}')

Score: None  Unexpected error: code 12345


## Get best result and configuration

In [6]:
task.get_best_result_and_configuration()

({ 'configuration': 'fda7e52d-220d-406b-98fc-8ac12b70b4f8',
   'id': 4458,
   'score': 1.5,
   'user_defined_data': None}, { 'id': 'fda7e52d-220d-406b-98fc-8ac12b70b4f8',
   'type': 'user-defined',
   'values': {'x': 1, 'y': 2}})

## Get top N results and configurations

In [7]:
for result in task.get_results(limit=3, best_first=True):
    configuration = task.get_configuration(result.configuration)
    display(f'Score: {result.score:.3f} for configuration: {configuration.values}')

"Score: 1.500 for configuration: {'x': 1, 'y': 2}"

"Score: 0.674 for configuration: {'x': 0.5900467432072032, 'y': 1.723369643705435}"

"Score: 0.417 for configuration: {'x': 0.5, 'y': 1.5}"

## Resume a completed task
Completing a task means that no further configurations can be generated and no further results can be recorded for it.

In [8]:
task.complete()
try:
    task.generate_configurations()
except Exception as err:
    print(err)

Cannot add configurations to a completed task


However, you can resume a completed task if necessary:

In [9]:
task.resume()
task.generate_configurations()

[{ 'id': '2afcf327-ffc2-4f7e-a075-dc43a0d7f022',
   'type': 'exploration',
   'values': {'x': 1.8852008532413678, 'y': 2.463925987958365}}]